In [29]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os

# Codon table for translation
codon_table = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'AGU': 'S', 'AGC': 'S', 'CCU': 'P', 'CCC': 'P',
    'CCA': 'P', 'CCG': 'P', 'ACU': 'T', 'ACC': 'T',
    'ACA': 'T', 'ACG': 'T', 'GCU': 'A', 'GCC': 'A',
    'GCA': 'A', 'GCG': 'A', 'UAU': 'Y', 'UAC': 'Y',
    'UAA': '*', 'UAG': '*', 'UGA': '*', 'CAU': 'H',
    'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'AAU': 'N',
    'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'GAU': 'D',
    'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'UGU': 'C',
    'UGC': 'C', 'UGG': 'W', 'CGU': 'R', 'CGC': 'R',
    'CGA': 'R', 'CGG': 'R', 'AGA': 'R', 'AGG': 'R',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

def reverse_complement(seq):
    """Generate the reverse complement of an RNA sequence."""
    complement = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}
    return "".join(complement[base] for base in reversed(seq))

def validate_sequence(sequence):
    """Check if the sequence contains only valid nucleotides (A, T, G, C, U)."""
    valid_bases = {'A', 'T', 'G', 'C', 'U'}
    return all(base in valid_bases for base in sequence)

def translate_sequence(sequence):
    """Translate a DNA or RNA sequence into a protein sequence."""
    sequence = sequence.replace(" ", "").upper()
    if not validate_sequence(sequence):
        return "Error: Invalid character found. Only 'A', 'T', 'G', 'C', 'U' are allowed."
    if 'T' in sequence:
        sequence = sequence.replace('T', 'U')
    
    protein = ""
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        amino_acid = codon_table.get(codon, '?')
        if amino_acid == '*':
            break
        protein += amino_acid
    return protein

def translate_frames(sequence):
    """Find all six reading frames and their translated protein sequences."""
    sequence = sequence.replace(" ", "").upper()
    if not validate_sequence(sequence):
        return "Error: Invalid character found. Only 'A', 'T', 'G', 'C', 'U' are allowed."
    if 'T' in sequence:
        sequence = sequence.replace('T', 'U')
    
    forward_frames = [sequence[i:] for i in range(3)]
    reverse_seq = reverse_complement(sequence)
    reverse_frames = [reverse_seq[i:] for i in range(3)]
    
    all_frames = forward_frames + reverse_frames
    frame_labels = ["Forward Frame 1", "Forward Frame 2", "Forward Frame 3",
                    "Reverse Frame 1", "Reverse Frame 2", "Reverse Frame 3"]
    
    results = {}
    for label, frame in zip(frame_labels, all_frames):
        protein = ""
        start_found = False
        for i in range(0, len(frame) - 2, 3):
            codon = frame[i:i+3]
            if codon == 'AUG':
                start_found = True
                protein = "M"
            elif start_found:
                if codon in {'UAA', 'UAG', 'UGA'}:
                    break
                protein += codon_table.get(codon, '?')
        results[label] = protein if protein else "No valid protein found"
    return results

def save_output_to_txt(output_text, input_file):
    """Saves the output to a text file in the same directory as the input file."""
    output_dir = os.path.dirname(input_file)
    output_file = os.path.join(output_dir, "translated_output.txt")
    with open(output_file, 'w') as file:
        file.write(output_text)
    print(f"Translation results saved to: {output_file}")

def read_fasta_or_sequence(file_path):
    """Reads a sequence from a FASTA or plain sequence file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if lines[0].startswith('>'):
            return ''.join(line.strip() for line in lines[1:])
        return ''.join(line.strip() for line in lines)

def browse_file():
    """Open a file dialog to select a file and load the sequence."""
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("FASTA files", "*.fasta"), ("All files", "*.*")])
    if file_path:
        try:
            with open(file_path, 'r') as file:
                sequence = file.read().strip()
                sequence_entry.delete(1.0, tk.END)  # Clear previous input
                sequence_entry.insert(tk.END, sequence)
        except Exception as e:
            messagebox.showerror("Error", f"Error reading the file: {e}")

def translate_sequence_gui():
    """Translate the sequence input using GUI."""
    sequence = sequence_entry.get("1.0", "end-1c").strip().replace(" ", "").upper()
    
    if not validate_sequence(sequence):
        messagebox.showerror("Invalid Sequence", "Sequence contains invalid characters. Only 'A', 'T', 'G', 'C', 'U' are allowed.")
        return
    
    forward = forward_var.get()
    reverse = reverse_var.get()

    if not forward and not reverse:
        messagebox.showwarning("No Direction", "Please select a translation direction (Forward or Reverse).")
        return
    
    translation_result = ""
    if forward:
        translation_result += f"Forward Translation:\n{translate_sequence(sequence)}\n\n"
    
    if reverse:
        reverse_seq = reverse_complement(sequence)
        translation_result += f"Reverse Translation:\n{translate_sequence(reverse_seq)}\n\n"

    output_text.delete(1.0, tk.END)  # Clear previous output
    output_text.insert(tk.END, translation_result)

def save_output():
    """Save the output text to a file."""
    output_text_content = output_text.get("1.0", tk.END).strip()
    if not output_text_content:
        messagebox.showwarning("No Output", "There is no output to save.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if file_path:
        try:
            with open(file_path, 'w') as file:
                file.write(output_text_content)
            messagebox.showinfo("Success", f"Output saved to {file_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Error saving the file: {e}")

# Create the main window
root = tk.Tk()
root.title("Translator (ORF Tool)")

# --- Input Area ---
input_frame = tk.LabelFrame(root, text="Enter DNA or RNA sequence")
input_frame.pack(padx=10, pady=10, fill="x")

sequence_entry = tk.Text(input_frame, height=5)
sequence_entry.pack(padx=5, pady=5, fill="x")

browse_button = tk.Button(input_frame, text="Browse...", command=browse_file)
browse_button.pack(side="right", padx=5, pady=5)

# --- Direction Options ---
direction_frame = tk.Frame(root)
direction_frame.pack(padx=10, pady=5, fill="x")

forward_var = tk.BooleanVar()
forward_check = tk.Checkbutton(direction_frame, text="Forward", variable=forward_var)
forward_check.pack(side="left")

reverse_var = tk.BooleanVar()
reverse_check = tk.Checkbutton(direction_frame, text="Reverse", variable=reverse_var)
reverse_check.pack(side="left", padx=10)

# --- Translate Button ---
translate_button = tk.Button(root, text="Translate", command=translate_sequence_gui)
translate_button.pack(pady=10)

# --- Output Area ---
output_frame = tk.LabelFrame(root, text="Output")
output_frame.pack(padx=10, pady=10, fill="both", expand=True)

output_text = tk.Text(output_frame, height=10, width=50)
output_text.pack(padx=5, pady=5, fill="both", expand=True)

# --- Save Output Button ---
save_button = tk.Button(root, text="Save Output", command=save_output)
save_button.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os

# Codon table for translation
codon_table = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'AGU': 'S', 'AGC': 'S', 'CCU': 'P', 'CCC': 'P',
    'CCA': 'P', 'CCG': 'P', 'ACU': 'T', 'ACC': 'T',
    'ACA': 'T', 'ACG': 'T', 'GCU': 'A', 'GCC': 'A',
    'GCA': 'A', 'GCG': 'A', 'UAU': 'Y', 'UAC': 'Y',
    'UAA': '*', 'UAG': '*', 'UGA': '*', 'CAU': 'H',
    'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'AAU': 'N',
    'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'GAU': 'D',
    'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'UGU': 'C',
    'UGC': 'C', 'UGG': 'W', 'CGU': 'R', 'CGC': 'R',
    'CGA': 'R', 'CGG': 'R', 'AGA': 'R', 'AGG': 'R',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

def reverse_complement(seq):
    complement = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}
    return "".join(complement[base] for base in reversed(seq))

def validate_sequence(sequence):
    valid_bases = {'A', 'T', 'G', 'C', 'U'}
    return all(base in valid_bases for base in sequence)

def find_all_orfs(seq):
    """Find multiple ORFs (from AUG to stop) in a given sequence."""
    proteins = []
    i = 0
    while i < len(seq) - 2:
        codon = seq[i:i+3]
        if codon == 'AUG':
            protein = 'M'
            j = i + 3
            while j < len(seq) - 2:
                next_codon = seq[j:j+3]
                if next_codon in {'UAA', 'UAG', 'UGA'}:
                    proteins.append(protein)
                    break
                protein += codon_table.get(next_codon, '?')
                j += 3
            i = j  # skip past this ORF
        else:
            i += 3
    return proteins if proteins else ["No valid protein found"]

def translate_frames(sequence):
    sequence = sequence.replace(" ", "").upper()
    if not validate_sequence(sequence):
        return {"Error": "Invalid character found. Only 'A', 'T', 'G', 'C', 'U' are allowed."}
    if 'T' in sequence:
        sequence = sequence.replace('T', 'U')

    frames = [sequence[i:] for i in range(3)]
    rev_seq = reverse_complement(sequence)
    frames += [rev_seq[i:] for i in range(3)]

    labels = [
        "Forward Frame 1", "Forward Frame 2", "Forward Frame 3",
        "Reverse Frame 1", "Reverse Frame 2", "Reverse Frame 3"
    ]

    results = {}
    for label, frame in zip(labels, frames):
        results[label] = find_all_orfs(frame)
    return results

def read_fasta_or_sequence(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        start = next((i for i, line in enumerate(lines) if line.startswith('>')), -1)
        if start != -1:
            return ''.join(line.strip() for line in lines[start+1:])
        return ''.join(line.strip() for line in lines)

def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("FASTA files", "*.fasta"), ("All files", "*.*")])
    if file_path:
        try:
            sequence = read_fasta_or_sequence(file_path)
            sequence_entry.delete(1.0, tk.END)
            sequence_entry.insert(tk.END, sequence)
        except Exception as e:
            messagebox.showerror("Error", f"Error reading the file: {e}")

def translate_sequence_gui():
    sequence = sequence_entry.get("1.0", "end-1c").strip().replace(" ", "").upper()

    if not validate_sequence(sequence):
        messagebox.showerror("Invalid Sequence", "Sequence contains invalid characters. Only 'A', 'T', 'G', 'C', 'U' are allowed.")
        return

    forward = forward_var.get()
    reverse = reverse_var.get()

    if not forward and not reverse:
        messagebox.showwarning("No Direction", "Please select a translation direction (Forward or Reverse).")
        return

    results = translate_frames(sequence)

    output_text.delete(1.0, tk.END)
    for label, orfs in results.items():
        if (forward and "Forward" in label) or (reverse and "Reverse" in label):
            output_text.insert(tk.END, f"{label}:\n")
            for idx, orf in enumerate(orfs, 1):
                output_text.insert(tk.END, f"  ORF {idx}: {orf}\n")
            output_text.insert(tk.END, "\n")

def save_output():
    content = output_text.get("1.0", tk.END).strip()
    if not content:
        messagebox.showwarning("No Output", "There is no output to save.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if file_path:
        try:
            with open(file_path, 'w') as file:
                file.write(content)
            messagebox.showinfo("Success", f"Output saved to {file_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Error saving the file: {e}")

# --- GUI Setup ---
root = tk.Tk()
root.title("Translator (ORF Tool)")

input_frame = tk.LabelFrame(root, text="Enter DNA or RNA sequence")
input_frame.pack(padx=10, pady=10, fill="x")

sequence_entry = tk.Text(input_frame, height=5)
sequence_entry.pack(padx=5, pady=5, fill="x")

browse_button = tk.Button(input_frame, text="Browse...", command=browse_file)
browse_button.pack(side="right", padx=5, pady=5)

direction_frame = tk.Frame(root)
direction_frame.pack(padx=10, pady=5, fill="x")

forward_var = tk.BooleanVar()
reverse_var = tk.BooleanVar()
tk.Checkbutton(direction_frame, text="Forward", variable=forward_var).pack(side="left")
tk.Checkbutton(direction_frame, text="Reverse", variable=reverse_var).pack(side="left", padx=10)

tk.Button(root, text="Translate", command=translate_sequence_gui).pack(pady=10)

output_frame = tk.LabelFrame(root, text="Output")
output_frame.pack(padx=10, pady=10, fill="both", expand=True)

output_text = tk.Text(output_frame, height=10, width=50)
output_text.pack(padx=5, pady=5, fill="both", expand=True)

tk.Button(root, text="Save Output", command=save_output).pack(pady=10)

root.mainloop()
